<a href="https://colab.research.google.com/github/Chaitra-07/Deep-learning/blob/master/Pro_sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import warnings
warnings.filterwarnings('ignore') 

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf


In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
%tensorflow_version 1.14

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
#!pip install tensorflow==1.14.0

In [0]:
tf.__version__

'1.14.0'

In [0]:
dataset = pd.read_csv('gdrive/My Drive/mydataset/sentitrain.csv')

In [0]:
dataset.shape

(7920, 3)

In [0]:
dataset.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [0]:
X =dataset.tweet
Y= dataset.label

In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)


In [0]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [0]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
______

In [0]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 5385 samples, validate on 1347 samples
Epoch 1/10
5385/5385 [==============================] - 11s 2ms/step - loss: 0.4056 - acc: 0.7989 - val_loss: 0.2572 - val_acc: 0.8946
Epoch 2/10
5385/5385 [==============================] - 10s 2ms/step - loss: 0.2549 - acc: 0.8908 - val_loss: 0.2368 - val_acc: 0.8968
Epoch 3/10
5385/5385 [==============================] - 10s 2ms/step - loss: 0.2250 - acc: 0.9018 - val_loss: 0.2388 - val_acc: 0.9050


In [0]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [0]:
accr = model.evaluate(test_sequences_matrix,Y_test)

1188/1188 [==============================] - 1s 674us/step


In [0]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


Test set
  Loss: 0.261
  Accuracy: 0.890


In [0]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test,test_sequences_matrix))
print(classification_report(Y_test,test_sequences_matrix))

ValueError: ignored

In [0]:
sample_dataset = pd.read_csv('gdrive/My Drive/mydataset/sentitest.csv')
#sample submission file

In [0]:
sample_dataset.shape

(1953, 2)

In [0]:
sample_dataset.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [0]:
X_sam = sample_dataset.tweet

In [0]:
tok.fit_on_texts(X_sam)
sequences = tok.texts_to_sequences(X_sam)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [0]:
Y_sam = model.predict(sequences_matrix)

In [0]:
Y_sam

array([[0.01178846],
       [0.44068035],
       [0.52423686],
       ...,
       [0.1748138 ],
       [0.03201944],
       [0.02600466]], dtype=float32)

In [0]:
accr = model.evaluate(sequences_matrix,Y_sam)

1953/1953 [==============================] - 1s 628us/step


In [0]:
accr

[0.2454826111404089, 0.0]

In [0]:
print('Sample set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


Sample set
  Loss: 0.245
  Accuracy: 0.000
